<a href="https://colab.research.google.com/github/jeonghun9326/syrup/blob/submission/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 농산물 가격 예측을 위한 AI 모델 개발
- '2024 농산물 가격 예측 AI 경진대회'는 데이터와 AI 기술을 활용하여 농산물 가격 예측 능력을 향상시키는 것을 목표로 합니다.<br>  이 대회는 농업 분야의 복잡한 시계열 데이터를 효율적으로 분석하고 예측할 수 있는 AI 알고리즘 개발에 초점을 맞추고 있습니다. <br> <br>
- 이 대회의 궁극적 목적은 참가자들의 시계열 데이터 분석 및 예측 역량을 강화하고, <br> AI 기술이 실제 농산물 가격 예측과 관련 정책 결정에 어떻게 기여할 수 있는지 탐구하는 것입니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Library

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Hyperparameter Setting

In [ ]:
config = {
    "learning_rate": 2e-5,
    "epoch": 30,
    "batch_size": 64,
    "hidden_size": 64,
    "num_layers": 2,
    "output_size": 3
}

CFG = SimpleNamespace(**config)

품목_리스트 = ['건고추', '사과', '감자', '배', '깐마늘(국산)', '무', '상추', '배추', '양파', '대파']

# Define Function for Feature Engineering
- 타겟의 필터 조건을 제외한 메타데이터의 필터 조건은 참가자들 각자의 기준에 맞춰 자유롭게 사용가능
- 밑의 필터 조건은 임의로 제공하는 예시

In [ ]:

def process_data(raw_file, 산지공판장_file, 전국도매_file, 품목명, scaler=None):
    raw_data = pd.read_csv('/content/drive/MyDrive/매디치기업연계/농산물예측/open (1)/train/train.csv')
    산지공판장 = pd.read_csv('/content/drive/MyDrive/매디치기업연계/농산물예측/open (1)/train/TRAIN_산지공판장_2018-2021.csv')
    전국도매 = pd.read_csv('/content/drive/MyDrive/매디치기업연계/농산물예측/open (1)/train/TRAIN_전국도매_2018-2021.csv')

    # 타겟 및 메타데이터 필터 조건 정의
    conditions = {
    '감자': {
        'target': lambda df: (df['품종명'] == '감자 수미') & (df['거래단위'] == '20키로상자') & (df['등급'] == '상'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['감자'], '품종명': ['수미'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['감자'], '품종명': ['수미']}
    },
    '건고추': {
        'target': lambda df: (df['품종명'] == '화건') & (df['거래단위'] == '30 kg') & (df['등급'] == '상품'),
        '공판장': None,
        '도매': None
    },
    '깐마늘(국산)': {
        'target': lambda df: (df['거래단위'] == '20 kg') & (df['등급'] == '상품'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['마늘'], '품종명': ['깐마늘'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['마늘'], '품종명': ['깐마늘']}
    },
    '대파': {
        'target': lambda df: (df['품종명'] == '대파(일반)') & (df['거래단위'] == '1키로단') & (df['등급'] == '상'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['대파'], '품종명': ['대파(일반)'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['대파'], '품종명': ['대파(일반)']}
    },
    '무': {
        'target': lambda df: (df['거래단위'] == '20키로상자') & (df['등급'] == '상'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['무'], '품종명': ['기타무'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['무'], '품종명': ['무']}
    },
    '배추': {
        'target': lambda df: (df['거래단위'] == '10키로망대') & (df['등급'] == '상'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['배추'], '품종명': ['쌈배추'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['배추'], '품종명': ['배추']}
    },
    '사과': {
        'target': lambda df: (df['품종명'].isin(['홍로', '후지'])) & (df['거래단위'] == '10 개') & (df['등급'] == '상품'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['사과'], '품종명': ['후지'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['사과'], '품종명': ['후지']}
    },
    '상추': {
        'target': lambda df: (df['품종명'] == '청') & (df['거래단위'] == '100 g') & (df['등급'] == '상품'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['상추'], '품종명': ['청상추'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['상추'], '품종명': ['청상추']}
    },
    '양파': {
        'target': lambda df: (df['품종명'] == '양파') & (df['거래단위'] == '1키로') & (df['등급'] == '상'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['양파'], '품종명': ['기타양파'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['양파'], '품종명': ['양파(일반)']}
    },
    '배': {
        'target': lambda df: (df['품종명'] == '신고') & (df['거래단위'] == '10 개') & (df['등급'] == '상품'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['배'], '품종명': ['신고'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['배'], '품종명': ['신고']}
    }
    }

    # 타겟 데이터 필터링
    raw_품목 = raw_data[raw_data['품목명'] == 품목명]
    target_mask = conditions[품목명]['target'](raw_품목)
    filtered_data = raw_품목[target_mask]

    # 다른 품종에 대한 파생변수 생성
    other_data = raw_품목[~target_mask]
    unique_combinations = other_data[['품종명', '거래단위', '등급']].drop_duplicates()
    for _, row in unique_combinations.iterrows():
        품종명, 거래단위, 등급 = row['품종명'], row['거래단위'], row['등급']
        mask = (other_data['품종명'] == 품종명) & (other_data['거래단위'] == 거래단위) & (other_data['등급'] == 등급)
        temp_df = other_data[mask]
        for col in ['평년 평균가격(원)', '평균가격(원)']:
            new_col_name = f'{품종명}_{거래단위}_{등급}_{col}'
            filtered_data = filtered_data.merge(temp_df[['시점', col]], on='시점', how='left', suffixes=('', f'_{new_col_name}'))
            filtered_data.rename(columns={f'{col}_{new_col_name}': new_col_name}, inplace=True)


    # 공판장 데이터 처리
    if conditions[품목명]['공판장']:
        filtered_공판장 = 산지공판장
        for key, value in conditions[품목명]['공판장'].items():
            filtered_공판장 = filtered_공판장[filtered_공판장[key].isin(value)]

        filtered_공판장 = filtered_공판장.add_prefix('공판장_').rename(columns={'공판장_시점': '시점'})
        filtered_data = filtered_data.merge(filtered_공판장, on='시점', how='left')

    # 도매 데이터 처리
    if conditions[품목명]['도매']:
        filtered_도매 = 전국도매
        for key, value in conditions[품목명]['도매'].items():
            filtered_도매 = filtered_도매[filtered_도매[key].isin(value)]

        filtered_도매 = filtered_도매.add_prefix('도매_').rename(columns={'도매_시점': '시점'})
        filtered_data = filtered_data.merge(filtered_도매, on='시점', how='left')

    # 수치형 컬럼 처리
    numeric_columns = filtered_data.select_dtypes(include=[np.number]).columns
    filtered_data = filtered_data[['시점'] + list(numeric_columns)]
    filtered_data[numeric_columns] = filtered_data[numeric_columns].fillna(0)

    # 정규화 적용
    if scaler is None:
        scaler = MinMaxScaler()
        filtered_data[numeric_columns] = scaler.fit_transform(filtered_data[numeric_columns])
    else:
        filtered_data[numeric_columns] = scaler.transform(filtered_data[numeric_columns])

    return filtered_data, scaler


# Define Custom Dataset Class

In [ ]:
class AgriculturePriceDataset(Dataset):
    def __init__(self, dataframe, window_size=9, prediction_length=3, is_test=False):
        self.data = dataframe
        self.window_size = window_size
        self.prediction_length = prediction_length
        self.is_test = is_test

        self.price_column = '평균가격(원)'
        self.numeric_columns = self.data.select_dtypes(include=[np.number]).columns.tolist()

        self.sequences = []
        if not self.is_test:
            for i in range(len(self.data) - self.window_size - self.prediction_length + 1):
                x = self.data[self.numeric_columns].iloc[i:i+self.window_size].values
                y = self.data[self.price_column].iloc[i+self.window_size:i+self.window_size+self.prediction_length].values
                self.sequences.append((x, y))
        else:
            self.sequences = [self.data[self.numeric_columns].values]

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        if not self.is_test:
            x, y = self.sequences[idx]
            return torch.FloatTensor(x), torch.FloatTensor(y)
        else:
            return torch.FloatTensor(self.sequences[idx])

# Define Model Architecture and Training Functions

In [ ]:

class PricePredictionLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(PricePredictionLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    total_loss = 0
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def evaluate_model(model, test_loader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            total_loss += loss.item()
    return total_loss / len(test_loader)

# Train Models and Generate Predictions

In [ ]:
품목별_predictions = {}
품목별_scalers = {}

pbar_outer = tqdm(품목_리스트, desc="품목 처리 중", position=0)
for 품목명 in pbar_outer:
    pbar_outer.set_description(f"품목별 전처리 및 모델 학습 -> {품목명}")
    train_data, scaler = process_data("./train/train.csv",
                              "./train/meta/TRAIN_산지공판장_2018-2021.csv",
                              "./train/meta/TRAIN_전국도매_2018-2021.csv",
                              품목명)
    품목별_scalers[품목명] = scaler
    dataset = AgriculturePriceDataset(train_data)

    # 데이터를 train과 validation으로 분할
    train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42)

    train_loader = DataLoader(train_data, CFG.batch_size, shuffle=True)
    val_loader = DataLoader(val_data, CFG.batch_size, shuffle=False)

    input_size = len(dataset.numeric_columns)

    model = PricePredictionLSTM(input_size, CFG.hidden_size, CFG.num_layers, CFG.output_size)
    criterion = nn.L1Loss()
    optimizer = torch.optim.Adam(model.parameters(), CFG.learning_rate)

    best_val_loss = float('inf')
    os.makedirs('models', exist_ok=True)

    for epoch in range(CFG.epoch):
        train_loss = train_model(model, train_loader, criterion, optimizer, CFG.epoch)
        val_loss = evaluate_model(model, val_loader, criterion)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), f'models/best_model_{품목명}.pth')

        print(f'Epoch {epoch+1}/{CFG.epoch}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

    print(f'Best Validation Loss for {품목명}: {best_val_loss:.4f}')

    품목_predictions = []

    ### 추론
    pbar_inner = tqdm(range(25), desc="테스트 파일 추론 중", position=1, leave=False)
    for i in pbar_inner:
        test_file = f"./test/TEST_{i:02d}.csv"
        산지공판장_file = f"./test/meta/TEST_산지공판장_{i:02d}.csv"
        전국도매_file = f"./test/meta/TEST_전국도매_{i:02d}.csv"

        test_data, _ = process_data(test_file, 산지공판장_file, 전국도매_file, 품목명, scaler=품목별_scalers[품목명])
        test_dataset = AgriculturePriceDataset(test_data, is_test=True)
        test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

        model.eval()
        predictions = []
        with torch.no_grad():
            for batch in test_loader:
                output = model(batch)
                predictions.append(output.numpy())

        predictions_array = np.concatenate(predictions)

        # 예측값을 원래 스케일로 복원
        price_column_index = test_data.columns.get_loc(test_dataset.price_column)
        predictions_reshaped = predictions_array.reshape(-1, 1)

        # 가격 열에 대해서만 inverse_transform 적용
        price_scaler = MinMaxScaler()
        price_scaler.min_ = 품목별_scalers[품목명].min_[price_column_index]
        price_scaler.scale_ = 품목별_scalers[품목명].scale_[price_column_index]
        predictions_original_scale = price_scaler.inverse_transform(predictions_reshaped)
        #print(predictions_original_scale)

        if np.isnan(predictions_original_scale).any():
            pbar_inner.set_postfix({"상태": "NaN"})
        else:
            pbar_inner.set_postfix({"상태": "정상"})
            품목_predictions.extend(predictions_original_scale.flatten())


    품목별_predictions[품목명] = 품목_predictions
    pbar_outer.update(1)


품목 처리 중:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1/30, Train Loss: 0.4712, Val Loss: 0.3848
Epoch 2/30, Train Loss: 0.4666, Val Loss: 0.3839
Epoch 3/30, Train Loss: 0.4672, Val Loss: 0.3830
Epoch 4/30, Train Loss: 0.4660, Val Loss: 0.3821
Epoch 5/30, Train Loss: 0.4575, Val Loss: 0.3812
Epoch 6/30, Train Loss: 0.4562, Val Loss: 0.3803
Epoch 7/30, Train Loss: 0.4624, Val Loss: 0.3794
Epoch 8/30, Train Loss: 0.4607, Val Loss: 0.3786
Epoch 9/30, Train Loss: 0.4644, Val Loss: 0.3777
Epoch 10/30, Train Loss: 0.4619, Val Loss: 0.3768
Epoch 11/30, Train Loss: 0.4648, Val Loss: 0.3759
Epoch 12/30, Train Loss: 0.4540, Val Loss: 0.3750
Epoch 13/30, Train Loss: 0.4591, Val Loss: 0.3742
Epoch 14/30, Train Loss: 0.4611, Val Loss: 0.3733
Epoch 15/30, Train Loss: 0.4476, Val Loss: 0.3724
Epoch 16/30, Train Loss: 0.4562, Val Loss: 0.3715
Epoch 17/30, Train Loss: 0.4543, Val Loss: 0.3706
Epoch 18/30, Train Loss: 0.4543, Val Loss: 0.3697
Epoch 19/30, Train Loss: 0.4524, Val Loss: 0.3689
Epoch 20/30, Train Loss: 0.4468, Val Loss: 0.3680
Epoch 21/

테스트 파일 추론 중:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 1/30, Train Loss: 0.3827, Val Loss: 0.3210
Epoch 2/30, Train Loss: 0.3919, Val Loss: 0.3200
Epoch 3/30, Train Loss: 0.3953, Val Loss: 0.3190
Epoch 4/30, Train Loss: 0.3907, Val Loss: 0.3180
Epoch 5/30, Train Loss: 0.4037, Val Loss: 0.3170
Epoch 6/30, Train Loss: 0.3885, Val Loss: 0.3160
Epoch 7/30, Train Loss: 0.3930, Val Loss: 0.3150
Epoch 8/30, Train Loss: 0.3997, Val Loss: 0.3140
Epoch 9/30, Train Loss: 0.3890, Val Loss: 0.3130
Epoch 10/30, Train Loss: 0.3921, Val Loss: 0.3121
Epoch 11/30, Train Loss: 0.3835, Val Loss: 0.3111
Epoch 12/30, Train Loss: 0.3879, Val Loss: 0.3101
Epoch 13/30, Train Loss: 0.3860, Val Loss: 0.3091
Epoch 14/30, Train Loss: 0.3864, Val Loss: 0.3081
Epoch 15/30, Train Loss: 0.3785, Val Loss: 0.3072
Epoch 16/30, Train Loss: 0.3813, Val Loss: 0.3062
Epoch 17/30, Train Loss: 0.3752, Val Loss: 0.3052
Epoch 18/30, Train Loss: 0.3851, Val Loss: 0.3042
Epoch 19/30, Train Loss: 0.3949, Val Loss: 0.3032
Epoch 20/30, Train Loss: 0.3803, Val Loss: 0.3022
Epoch 21/

테스트 파일 추론 중:   0%|          | 0/25 [00:00<?, ?it/s]

...

테스트 파일 추론 중:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 1/30, Train Loss: 0.2749, Val Loss: 0.2197
Epoch 2/30, Train Loss: 0.2708, Val Loss: 0.2188
Epoch 3/30, Train Loss: 0.2655, Val Loss: 0.2179
Epoch 4/30, Train Loss: 0.2652, Val Loss: 0.2170
Epoch 5/30, Train Loss: 0.2668, Val Loss: 0.2161
Epoch 6/30, Train Loss: 0.2619, Val Loss: 0.2152
Epoch 7/30, Train Loss: 0.2636, Val Loss: 0.2143
Epoch 8/30, Train Loss: 0.2665, Val Loss: 0.2134
Epoch 9/30, Train Loss: 0.2643, Val Loss: 0.2125
Epoch 10/30, Train Loss: 0.2549, Val Loss: 0.2116
Epoch 11/30, Train Loss: 0.2569, Val Loss: 0.2107
Epoch 12/30, Train Loss: 0.2580, Val Loss: 0.2098
Epoch 13/30, Train Loss: 0.2533, Val Loss: 0.2089
Epoch 14/30, Train Loss: 0.2611, Val Loss: 0.2080
Epoch 15/30, Train Loss: 0.2555, Val Loss: 0.2071
Epoch 16/30, Train Loss: 0.2504, Val Loss: 0.2063
Epoch 17/30, Train Loss: 0.2523, Val Loss: 0.2054
Epoch 18/30, Train Loss: 0.2571, Val Loss: 0.2045
Epoch 19/30, Train Loss: 0.2486, Val Loss: 0.2036
Epoch 20/30, Train Loss: 0.2540, Val Loss: 0.2027
Epoch 21/

테스트 파일 추론 중:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 1/30, Train Loss: 0.2638, Val Loss: 0.2345
Epoch 2/30, Train Loss: 0.2625, Val Loss: 0.2333
Epoch 3/30, Train Loss: 0.2557, Val Loss: 0.2322
Epoch 4/30, Train Loss: 0.2616, Val Loss: 0.2311
Epoch 5/30, Train Loss: 0.2594, Val Loss: 0.2300
Epoch 6/30, Train Loss: 0.2527, Val Loss: 0.2288
Epoch 7/30, Train Loss: 0.2665, Val Loss: 0.2277
Epoch 8/30, Train Loss: 0.2536, Val Loss: 0.2266
Epoch 9/30, Train Loss: 0.2591, Val Loss: 0.2255
Epoch 10/30, Train Loss: 0.2506, Val Loss: 0.2243
Epoch 11/30, Train Loss: 0.2532, Val Loss: 0.2232
Epoch 12/30, Train Loss: 0.2499, Val Loss: 0.2221
Epoch 13/30, Train Loss: 0.2468, Val Loss: 0.2210
Epoch 14/30, Train Loss: 0.2510, Val Loss: 0.2198
Epoch 15/30, Train Loss: 0.2466, Val Loss: 0.2187
Epoch 16/30, Train Loss: 0.2485, Val Loss: 0.2176
Epoch 17/30, Train Loss: 0.2458, Val Loss: 0.2164
Epoch 18/30, Train Loss: 0.2476, Val Loss: 0.2154
Epoch 19/30, Train Loss: 0.2482, Val Loss: 0.2143
Epoch 20/30, Train Loss: 0.2371, Val Loss: 0.2132
Epoch 21/

테스트 파일 추론 중:   0%|          | 0/25 [00:00<?, ?it/s]

# Prepare Submission File

In [ ]:
sample_submission = pd.read_csv('./sample_submission.csv')

for 품목명, predictions in 품목별_predictions.items():
    sample_submission[품목명] = predictions

# 결과 저장
sample_submission.to_csv('./baseline_submission.csv', index=False)
